In [1]:
n_cores = -1

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.cluster import KMeans

In [4]:
data = np.load('inga_out.npz')

In [5]:
mass = data['arr_0']
mass

array([ 552.293448,  338.046024,  724.221464, ...,  328.072907,
        262.19328 ,  636.278192])

In [6]:
name = data['arr_1']
name

array(['UNPD98266', 'UNPD207163', 'UNPD3499', ..., 'UNPD98267',
       'UNPD47332', 'UNPD101003'], dtype=object)

In [7]:
massabund = data['arr_2']
massabund

array([[  41.00329 ,    1.688456],
       [  43.01894 ,    2.135631],
       [  55.01894 ,    1.105409],
       ..., 
       [ 549.23414 ,   24.377134],
       [ 551.24979 ,   22.666363],
       [ 591.2447  ,    5.496205]])

In [8]:
blockind = data['arr_3']
blockind

array([     0,      0,      0, ..., 220988, 220988, 220988], dtype=uint32)

### Clustering

In [9]:
def clustering(name, massabund, blockind, n_clusters):
    values = massabund[:,0]
    kmeans = KMeans(n_clusters=n_clusters, n_jobs=n_cores).fit(values.reshape(-1,1))
    labels = kmeans.labels_
    
    features = generateMatrix(massabund, blockind, labels)
    np.save('clustering_'+str(n_clusters), features)
    
#     result = generateDataFrame(name, features)
#     result.to_csv('clustering_'+str(n)+'.csv', index=None)
    
    return features

In [10]:
def generateMatrix(massabund, blockind, labels):
    n_blocks = blockind[-1] + 1
    n_points = labels.size
    n_clusters = np.unique(labels).size
    result = np.zeros((n_blocks, n_clusters))
    previous = 0
    for i in range(n_points):
        if blockind[i] != previous:
            previous = blockind[i]
        result[previous, labels[i]] = result[previous, labels[i]] + massabund[i,1]
    return result

In [11]:
def generateDataFrame(name, features):
    mat = np.concatenate([name.reshape(-1,1), features], axis=1)
    column_name = ['name']
    for i in range(features.shape[1]):
        column_name.append('fraction'+str(i+1))
    df = pd.DataFrame(mat, columns=column_name)
    return df

In [12]:
for n in [300, 400, 500]:
    clustering(name, massabund, blockind, n)